In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy

In [2]:
analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load('en')

In [22]:
test_sentence = 'I\'m not sure why Lindsey Graham still gets as much attention as he does. He used to be a fairly interesting Republican, but these days he\'s just Tom Cotton with a personality.'

In [23]:
doc = nlp(test_sentence)
subtokens = [token for token in doc] 

In [24]:
print([toc.dep_ for toc in subtokens])

['nsubj', 'ROOT', 'neg', 'acomp', 'advmod', 'compound', 'nsubj', 'advmod', 'ccomp', 'advmod', 'amod', 'dobj', 'mark', 'nsubj', 'advcl', 'punct', 'nsubj', 'ROOT', 'aux', 'xcomp', 'det', 'advmod', 'amod', 'attr', 'punct', 'cc', 'det', 'npadvmod', 'nsubj', 'conj', 'advmod', 'compound', 'attr', 'prep', 'det', 'pobj', 'punct']


In [25]:
print(subtokens)

[I, 'm, not, sure, why, Lindsey, Graham, still, gets, as, much, attention, as, he, does, ., He, used, to, be, a, fairly, interesting, Republican, ,, but, these, days, he, 's, just, Tom, Cotton, with, a, personality, .]


In [26]:
def analyze(sentence):
    subject_scores = {}
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'nsubj':
            scores = analyzer.polarity_scores(token.sent.text)
            score = scores['compound']
            subject_scores[token] = score * 5 + 5
    return subject_scores
    

In [27]:
analyze(test_sentence)

{I: 3.7945, Graham: 3.7945, he: 3.7945, He: 6.072, he: 6.072}